# SDO Machine Learning (SDO ML) Dataset

In this notebook we demonstrate the process for interacting with a small sample of the SDO ML dataset.

*Meng Jin, Mark Cheung, & Paul Wright*

---

## Introduction

### SDO Overview

Since its launch in 2010, NASA’s Solar Dynamics Observatory (SDO; ([Pesnell et al. 2012](https://ui.adsabs.harvard.edu/abs/2012SoPh..275....3P/abstract)) has continuously monitored Sun's activity, delivering a wealth of valuable scientific data for heliophysics researchers with the use of three instruments:

1. The Atmospheric Imaging Assembly (AIA; [Lemen et al. 2012](https://ui.adsabs.harvard.edu/abs/2012SoPh..275...17L/abstract)), which captures 4096 x 4096 resolution images (with 0.6 arcsecond pixel size) of the full Sun in two ultraviolet (centered at 1600, and 1700 Å), seven extreme ultraviolet (EUV; centered at 94, 131, 171, 193, 211, 304, and 335 Å), and one visible (centered at 4500 Å) wavelength band.


2. The Helioseismic and Magnetic Imager (HMI; [Schou et al. 2012](https://ui.adsabs.harvard.edu/abs/2012SoPh..275..229S/abstract)) captures visible wavelength filtergrams of the full Sun at 4096 x 4096 resolution (a pixel size of 0.5 arcsecond), which are then processed into a number of data products, including photospheric Dopplergrams, line-of-sight magnetograms, and vector magnetograms ([Hoeksema et al. 2014](https://ui.adsabs.harvard.edu/abs/2014SoPh..289.3483H/abstract)).


3. The EUV Variability Experiment (EVE; [Woods et al. 2012](https://ui.adsabs.harvard.edu/abs/2012SoPh..275..115W/abstract)) monitors the solar EUV spectral irradiance from 1 to 1050 Å. This is done by utilizing multiple EUV Grating Spectrographs (MEGS) that disperse EUV light from the full disk of the Sun and its corona onto a 1024 x 2048 charge coupled device (CCD).

<img src="spacecraft_detailed.png" width="400">

**Figure 1:** The Solar Dynamics Observatory (SDO) spacecraft, shown with the three main instruments (AIA, EVE, and HMI) highlighted. Image courtesy of NASA (https://sdo.gsfc.nasa.gov).

### The SDO ML Dataset
The SDO ML Dataset (covering 2010 - 2018) was originally published as [Galvez et al (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..242....7G/abstract), and is hosted on the Stanford Digital Repository in Numpy's compressed array format (.npz). 

In this notebook, we present an update to the work outlined in Galvez et al (2019), in which the full dataset has been converted to cloud friendly Zarr format. Specifically, SDO/AIA data has been updated to account for a change in calibration after 2019. In addtion to the change in calibration, this updated format includes:

1. FITS header/keyword information (such as observation time, and exposure time).

2. processes for continually updating the data until the present day.

### Who is the SDO ML Dataset for?

The sheer volume of structured scientific data recorded by SDO (over 18 PB, and counting) is ideal for a range machine learning tasks (from time-series, to computer vision), as well as more traditional approaches.

While the level 1 data are easily accessible, pre-processing these data for scientific analysis often requires  specialized heliophysics (and instrument-specific) knowledge. This may act as an unnecessary hurdle for non-heliophysics machine learning researchers who may wish to experiment with datasets from the physical sciences, but are unaware of domain-specific nuances (e.g., that images must be spatially and temporally adjusted).

This notebook demonstrates the process for interacting with a subset of the curated SDO (AIA, HMI, EVE) dataset, that is mission-ready for machine-learning applications. **Our aim is to supply this standardized dataset for heliophysicists who wish to use machine learning in their own research, as well as machine-learning researchers who wish to develop models specialized for the physical sciences.**

---

## Table of Contents

The notebook is set out as follows:

1. Setting up the notebook
2. Reading and loading the data <br>
2a. Selecting images based on header information <br>
    2b. Selecting a subset of the data based on index <br>
    2c. Downsampling the data (resolution)<br>
    2d. Downsampling the data (temporally)<br>
3. Generating a SunPy map
4. Animating AIA data
5. Reading and loading HMI data
6. Reading and loading EVE data

## 1. Setting up the notebook

In [ ]:
!pip install zarr sunpy[all] gcsfs

In [ ]:
%matplotlib inline

import gcsfs
import zarr
import sunpy.map

import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import sunpy.visualization.colormaps as cm

from astropy.time import Time
from sunpy.visualization import axis_labels_from_ctype, wcsaxes_compat

from matplotlib import animation
from IPython.display import HTML

In [ ]:
# this is currently fine for interacting with notebooks from within the project
gcs = gcsfs.GCSFileSystem(access="read_only")
loc = "fdl-sdoml-v2/sdomlv2_small.zarr/"
store = gcsfs.GCSMap(loc, gcs=gcs, check=False)

## 2. Reading and loading the AIA data


The SDO ML dataset is stored in the Zarr format, a format for the storage of chunked, compressed, N-dimensional arrays with Numpy dtype. For an in-depth overview, see https://zarr.readthedocs.io/en/stable/tutorial.html.

In [ ]:
# first, we create a group with the store data located on GCP.
root = zarr.group(store)

In [ ]:
# Using `root.tree()`, we are able to display the hierarchy (of `loc`).
print(root.tree())

As shown in the tree, the heirachy consists of groups, each shown with their respective shape, and data type. In this example, we will primarily look at the 171 Å channel from 2010. This consists of 6135 512x512 images, stored as float32, and can be accessed as follows:

In [ ]:
data = root["2010"]["171A"]

We could have alternatively accessed the 2010 data as:

```
loc = 'fdl-sdoml-v2/sdomlv2_small.zarr/2010'
```

which becomes increasingly useful in the full dataset (where the heirachy contains years 2010 - present).

**Loading with Dask**

We can then load this data into an array using dask.

In [ ]:
all_image = da.from_array(data)
all_image

As shown above, the data has the shape (6135, 512, 512), and is split into 52 chunks of (120, 512, 512), each of 125.83 MB; this is further visualised on the right. The data is now in a form to be manipulated like a Numpy array.

We can load and display one image now:

In [ ]:
image=all_image[6000,:,:]
plt.figure(figsize=(10,10))
colormap = plt.get_cmap('sdoaia171')
plt.imshow(image,origin='lower',vmin=10,vmax=1000,cmap=colormap)

Depending on the use-case, we may wish to extract a subset of this data in various ways. In the following sections we step through a number of potential operations that we may wish to make with the data.

### 2a. Selecting images based on header information

The new data includes all fits header information with the same keywords. To find out the AIA keyword definition, one can refer to the following online document: 
http://jsoc.stanford.edu/~jsoc/keywords/AIA/AIA02840_K_AIA-SDO_FITS_Keyword_Document.pdf 

And one can list all the AIA keywords included:

In [ ]:
sorted(data.attrs)

We can extract the exposure (and observation) time from the data attributes (the header information), and downsample our data based upon that information.

In [ ]:
exptime = np.array(data.attrs["EXPTIME"])
t_obs = np.array(data.attrs["T_OBS"])

In [ ]:
plt.plot(exptime)

In [ ]:
# select indices where the exposure time is less than 2 seconds
index = np.where(exptime < 2.0)
selected_images = da.from_array(data)[index[0], :, :]

In [ ]:
selected_images

### 2b. Selecting images based on indices

While the data is not currently ordered by observation time, we can simple index the array to extract a number of observations

In [ ]:
# select a subset of the data based on index
selected_images = da.from_array(data)[100:200, :, :]
selected_header = {keys: values[100:200] for keys, values in data.attrs.items()}

# and even compute the mean of the selected images
print(f"mean of selected images: {da.mean(selected_images).compute():.2f}")

In [ ]:
selected_images

### 2c. Downsampling the data (resolution)

Downsample the 512x512 images to 256x256

In [ ]:
# downsample the data to 256 by sampling every other pixel
sub_index = np.arange(0, 512, 2)

sub_image = da.from_array(data)[0:600, sub_index, :]
sub_image = sub_image[:, :, sub_index]
# is this becuase dask doesn't support nd fancy indexing?

print(f"mean of selected images: {da.mean(sub_image).compute():.2f}")

In [ ]:
sub_image

### 2d. Downsampling the data (temporally)

We can use pandas to downstample the data within a time-range. Here, we choose 1 day of observations at 12 minute frequency

In [ ]:
df_time = pd.DataFrame(t_obs, index=np.arange(np.shape(t_obs)[0]), columns=["Time"])
df_time["Time"] = pd.to_datetime(df_time["Time"])

# select times at a frequency of 12 minutes
selected_times = pd.date_range(
    start="2010-08-28 00:00:00", end="2010-08-28 23:59:59", freq="12T", tz="UTC"
)

In [ ]:
selected_index = []
for i in selected_times:
    selected_index.append(np.argmin(abs(df_time["Time"] - i)))
# may be a more efficient way of doing this
time_index = list(filter(lambda x: x > 0, selected_index))
da.from_array(data)[time_index, :, :]

## 3. Generating a SunPy Map

SunPy is an open-source Python library for Solar Physics data analysis and visualization. 

In this section we will demonstrate how SunPy’s [Map](https://docs.sunpy.org/en/stable/api/sunpy.map.Map.html#sunpy.map.Map) with the Zarr-formatted data. We demonstrate this for a single index.

In [ ]:
# specify an image index
img_index = time_index[-1]

In [ ]:
# select the respective image, and header required for sunpy.map.Map()
selected_image = da.from_array(data)[img_index, :, :]
selected_headr = {keys: values[img_index] for keys, values in data.attrs.items()}

In [ ]:
my_map = sunpy.map.Map((np.array(selected_image), selected_headr))

In [ ]:
plt.figure(figsize=(7, 7))
ax = plt.subplot(projection=my_map)
my_map.plot()

As this is then a SunPy Map object, we can manipulate it as such.

In [ ]:
my_map

For more information about SunPy, and Map, see the SunPy project: https://readthedocs.org/projects/sunpy/

## 4. Generating an AIA animation

In [ ]:
maps = []

for i in time_index:
    selected_image = da.from_array(data)[i, :, :]
    selected_headr = {keys: values[i] for keys, values in data.attrs.items()}
    maps.append(sunpy.map.Map((np.array(selected_image), selected_headr)))

sq = sunpy.map.Map(maps, sequence=True, sortby="date")

In [ ]:
def animate_sequence(sequence, vmin = None, vmax = None):
    """
    An animation plotting routine to animate each element in MapSequence

    Parameters
    ----------
    sequence : `sunpy.map.mapsequence.MapSequence`
        a set of sunpy maps to animate

    Returns
    -------
    `matplotlib.animation.FuncAnimation`
        A FuncAnimation instance.
    """

    # fig, ax = plt.subplots(figsize=(7,7),
    #                       subplot_kw=dict(projection=sequence[0].wcs)
    #                      )

    ax = wcsaxes_compat.gca_wcs(sequence[0].wcs)
    fig = ax.get_figure()
    fig.figsize = (7, 7)

    plot_obj = sequence[0].plot(vmin = vmin, vmax = vmax)
    ax.set_title(sequence[0].latex_name)

    cbar = plt.colorbar()

    def update_fig(i):
        ax.set_title(sequence[i].latex_name)

        # set the data to that of the image being plot
        # Each image should be scaled as the first.
        plot_obj.set_data(sequence[i].data)

        # reset the WCS to that of the image being plot
        plot_obj.axes.reset_wcs(sequence[i].wcs)

        wcsaxes_compat.default_wcs_grid(ax)
        ax.set_xlabel(
            axis_labels_from_ctype(
                sequence[i].coordinate_system[0], sequence[i].spatial_units[0]
            )
        )
        ax.set_ylabel(
            axis_labels_from_ctype(
                sequence[i].coordinate_system[1], sequence[i].spatial_units[1]
            )
        )

        # basic test:
        # expect the wcs of the plot_obj to follow the sequence
        # print(sequence[i].wcs == plot_obj.axes.wcs, sequence[0].wcs == sequence[i].wcs)
        return (plot_obj,)

    anim = animation.FuncAnimation(
        fig, update_fig, init_func=None, frames=len(sequence), interval=60, repeat=True
    )
    plt.close(fig)

    return anim

In [ ]:
anim = animate_sequence(sq)
HTML(anim.to_jshtml())

## 5. Reading and loading HMI data

Now, we demonstrste how to read the HMI data within the SDOML dataset. In general, the HMI data structure is very similar to the AIA data. Instead of 9 AIA channels, for each time step, HMI has 3 channels that represent the 3 component of the vector magnetic field (i.e., Bx, By, and Bz). Using the same method described in section 2, we can load the HMI data in 2010:

In [ ]:
loc_hmi = "fdl-sdoml-v2/sdomlv2_hmi.zarr/2010"
store = gcsfs.GCSMap(loc_hmi, gcs=gcs, check=False)
root = zarr.group(store)
print(root.tree())

Same as AIA data, the new HMI data also includes all fits header information with the same keywords. To find out the HMI keyword definition, one can refer to the following online document:
http://jsoc.stanford.edu/doc/keywords/JSOC_Keywords_for_metadata.pdf

And one can list all the HMI keywords included:

In [ ]:
data = root["Bz"]
sorted(data.attrs)

Note that the HMI keywords are slightly different comparing with AIA keywords. For example, there is no exposure time ('EXPTIME') in the HMI metadata. Also, the 'T_OBS' format is a bit different:

In [ ]:
t_obs = np.array(data.attrs["T_OBS"])
t_obs

To keep it consistent with the AIA metadata, one can simply convert the time format:

In [ ]:
t_obs_new = []
for string in t_obs:
    new_string = string.replace('_TAI', 'Z')
    new_string = new_string.replace('_','T')
    new_string = new_string.replace('.','-')
    new_string = new_string.replace('Z', '.00')
    t_obs_new.append(new_string)
t = Time(t_obs_new, format='isot', scale='tai')
t_obs_new=t.value

Now we can load a magnetic map using Sunpy Map:

In [ ]:
da.from_array(data)[0, :, :]

In [ ]:
selected_headr

In [ ]:
selected_image = da.from_array(data)[0, :, :]
selected_headr = {keys: values[0] for keys, values in data.attrs.items()}
selected_headr['WAVEUNIT'] = 'angstrom'
my_map = sunpy.map.Map((np.array(selected_image), selected_headr))
my_map

We can load a movie of HMI Bz component in the same time range as shown for AIA in the previous section:

In [ ]:
df_time = pd.DataFrame(t_obs_new, index=np.arange(np.shape(t_obs_new)[0]), columns=["Time"])
df_time["Time"] = pd.to_datetime(df_time["Time"])

# select times at a frequency of 12 minutes
selected_times = pd.date_range(
    start="2010-08-28 00:00:00", end="2010-08-28 23:59:59", freq="12T")

selected_index = []
for i in selected_times:
    selected_index.append(np.argmin(abs(df_time["Time"] - i)))

#mark the missing timestamps in the data
missing_index=np.where(abs(df_time['Time'][selected_index]-selected_times)>pd.Timedelta('3m'))[0].tolist()
for i in missing_index:
    selected_index[i]=-1
        
# may be a more efficient way of doing this
time_index = list(filter(lambda x: x > 0, selected_index))
da.from_array(data)[time_index, :, :]

In [ ]:
maps = []

for i in time_index:
    selected_image = da.from_array(data)[i, :, :]
    selected_headr = {keys: values[i] for keys, values in data.attrs.items()}
    selected_headr['WAVEUNIT'] = 'angstrom'
    maps.append(sunpy.map.Map((np.array(selected_image), selected_headr)))

sq = sunpy.map.Map(maps, sequence=True, sortby="date")

In [ ]:
# The plot range can be specified by vmin and vmax
anim = animate_sequence(sq, vmin=-20, vmax=20)
HTML(anim.to_jshtml())

## 6. Reading and loading EVE data

The EVE data is timeseries data in 39 wavelengths from 2010 May 1 to 2014 May 26. The temporal resolution is 1 minute. In this section, we demonstrate how to read and load EVE data.

In [ ]:
loc_eve = "fdl-sdoml-v2/sdomlv2_eve.zarr"
store = gcsfs.GCSMap(loc_eve, gcs=gcs, check=False)
root = zarr.group(store)
print(root.tree())

As shown in the data structure tree, the ion line names can be identified. For some ions, there are multiple emission lines and the spectral line names end with '_1', '_2' to distinguish them. The details (e.g., ion, wavelength, emission temperature) are included in the metadata, which can be checked similar as AIA and HMI data:

In [ ]:
data = root["MEGS-A"]["Fe XX"]
sorted(data.attrs)

In [ ]:
data.attrs['ion'],data.attrs['wavelength'], data.attrs['logT']

In addition, the 'Time' variable contains all timestamps of the data:

In [ ]:
time = root['MEGS-A']['Time']
time[0:10]

We can plot the spectral line timeseries data:

In [ ]:
df = pd.Series(data,index=pd.to_datetime(time))
ax = df.plot(ylim=(0,2.e-4),figsize=(20,10),fontsize=20)
ax.set_title(str(data.attrs['ion'])+' '+str(data.attrs['wavelength'])+' LogT = '+str(data.attrs['logT']),fontsize=20)
ax.set_xlabel("Time",fontsize=20)
ax.set_ylabel("Intensity [W/m^2]",fontsize=20)

----

This project was conducted during the 2018 NASA Frontier Development Lab (FDL) program, a public/private partnership between NASA and SETI and industry partners including NVIDIA Corporation, Lockheed Martin, and IBM. The authors thank IBM (especially Naeem Altaf) for generously providing computing resources on the IBM Cloud.